# Исследование надёжности заёмщиков

Кредитному отделу банка нужно понять, влияет ли семейное положение и количество детей клиента на погашение кредита в срок.  Используем статистику о платёжеспособности клиентов.

Результаты исследования помогут при построении модели **кредитного скоринга**.

## Знакомство с данными

In [1]:
import pandas as pd

credit_info = pd.read_csv('data.csv')
# посмотрим общую информацию о данных
credit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Мы видим что второй и одинадцатый столбец имеют пропущенные значения. Еще эти столбцы имеют значения типа "float64" и будет удобнее и логичнее привести их к целочислительному типу.*

In [2]:
# посчитаем пропущенные значения
credit_info.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

*Пропущенные значения по доходу в точности совпадают по количеству с пропущенными значениями по стажу, предварительно можно сделать вывод о том что это заемщики без официального стажа и дохода - тем более 2174 строк с пропусками это около 10% всей информации!.*

In [3]:
# посмотрим первые строки таблицы
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


*Столбец 'days_employed' имеет отрицательные значения, а если перевести некоторые из них в года то получится стаж больше возраста заемщика, здесь можно поставить ограничение по возрасту с 18 лет (стаж не может превышать возраста заемщика минус 18 лет) с которых можно начинать официально трудиться.
Столбец 'education' имеет записи одинаковых значений разным шрифтом. Цели займа описаны разными словами, но имеют по сути одну цель - как в случае со свадьбой ("сыграть свадьбу" и "на проведение свадьбы").*

In [4]:
# посчитаем полные дубликаты
credit_info.duplicated().sum()

54

In [5]:
# посмотрим на данные с помощью метода 'describe()'
credit_info.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


*У нас 54 полных дубликата, и это еще без приведения одинаковых значений ячеек к общему формату. И возраст заемщика не может быть равным нулю, исправим это далее.*

**Вывод**

1. **В таблице есть пропуски которые логично заполнить нулями чтобы оставить данные о заемщиках без стажа и дохода.**
2. **Столбец с количеством стажа в днях можно привести к положительным, не превышающим возраста минус 18 лет заемщика значениям, а также исправить нулевой возраст**
3. **Типы обраования нужно привести к виду "lower_case" чтобы ['Среднее', 'СРЕДНЕЕ'] = 'среднее' и машина легко поняла бы что это одно и тоже.**
4. **Потребуется обрабатка целей займа для выявления одинаковых целей описанных разными словами.**

## Предобработка данных

### Обработка пропусков

In [6]:
# чтобы решить как нам заполнить пропуски взглянем на эти строки поближе
credit_info[credit_info['total_income'] == ""]

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


*Странно, как будто нет пустых строк, может дело в том что они не так уж пусты?*

In [7]:
# хоть для нас строки пусты, на самом деле они заполнены NaN, поэтому отбирать их стоит методом "isna()"
credit_info[pd.isna(credit_info['total_income'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


*Видно что NaN не зависит от конкретной группы заемщиков, предположим что это результат отсутствия справок о доходах и подтверждения стажа.*

In [8]:
# получается это не люди без дохода и стажа, а отсутствие информации у банка 
# давайте заполним пропуски средними значениями по столбцу income_type который нам понадобится далее
# для начала посмотрим какие группы заемщиков имеют пропуски в доходах
credit_info[pd.isna(credit_info['total_income']) == True].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,0,147,147,147,147,147,147,147,0,147
компаньон,508,0,508,508,508,508,508,508,508,0,508
пенсионер,413,0,413,413,413,413,413,413,413,0,413
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105


*Выделилось пять групп. Возможно для получения кредита этим группам заемщиков не требуется предоставлять справок о доходе и стаже (кроме предпринимателя), такие условия практикуются банками для доверенным клиентов.*

In [43]:
# расчитаем средний доход/стаж/возраст по группам
mean = credit_info[pd.isna(credit_info['total_income']) == False].groupby(
    'income_type').mean().sort_values('income_type').reset_index()
mean.loc[2:6, ['income_type','total_income', 'days_employed', 'dob_years']].style.hide_index()

income_type,total_income,days_employed,dob_years
госслужащий,170898,3349.82,40.7213
компаньон,202417,2094.53,39.7643
пенсионер,137127,14935.4,59.1426
предприниматель,499163,520,42.5
сотрудник,161380,2299.77,39.8977


*У пенсионеров средний стаж аномально большой, мы решим эту проблему далее. А пока посмотрим как расчитать средний стаж пенсионера. К примеру можно из его среднего возраста вычесть 18 лет и умножить на 365 дней. (59 - 18) * 365 =  14 965 дней, больше похоже на правду чем    365 003.*

In [10]:
# заменим пропуски на средние значения для заполненных ячеек по группам
credit_info.loc[(credit_info.income_type == 'госслужащий' ) & (
    pd.isna(credit_info.total_income)),'total_income'] = 170898.309923

credit_info.loc[(credit_info.income_type == 'компаньон' ) & (
    pd.isna(credit_info.total_income)),'total_income'] = 202417.461462

credit_info.loc[(credit_info.income_type == 'пенсионер' ) & (
    pd.isna(credit_info.total_income)),'total_income'] = 137127.465690

credit_info.loc[(credit_info.income_type == 'предприниматель' ) & (
    pd.isna(credit_info.total_income)),'total_income'] = 499163.144947

credit_info.loc[(credit_info.income_type == 'сотрудник' ) & (
    pd.isna(credit_info.total_income)),'total_income'] = 161380.260488

credit_info.loc[(credit_info.income_type == 'госслужащий' ) & (
    pd.isna(credit_info.days_employed)),'days_employed'] = 3399.896902

credit_info.loc[(credit_info.income_type == 'компаньон' ) & (
    pd.isna(credit_info.days_employed)),'days_employed'] = 2111.524398

credit_info.loc[(credit_info.income_type == 'пенсионер' ) & (
    pd.isna(credit_info.days_employed)),'days_employed'] = 14965.0

credit_info.loc[(credit_info.income_type == 'предприниматель' ) & (
    pd.isna(credit_info.days_employed)),'days_employed'] = 520.848083

credit_info.loc[(credit_info.income_type == 'сотрудник' ) & (
    pd.isna(credit_info.days_employed)),'days_employed'] = 2326.499216

credit_info.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

**Пропуски могли быть связаны с доверенными клиентами банка, в любом случае мы их заполнили наиболее близкими к выборке значениями по группам.**

### Замена типа данных

In [11]:
# выполним операции над столюцом 'days_employed'
# переведем отрицательные значения в положительные
credit_info['days_employed'] = credit_info['days_employed'].abs()
# проверим на отрицательные значения после обработки
is_negative = (credit_info[credit_info['days_employed'] < 0] ['days_employed'].count())
print('Количество отрицательных значений по столбцу после обработки: {}'.format(is_negative))

Количество отрицательных значений по столбцу после обработки: 0


In [12]:
# еще раз взглянем на таблицу
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [13]:
# приведем столбцы 'days_employed', 'total_income' к целочислительному типу и проверим результат
credit_info[['days_employed', 'total_income']] = credit_info[['days_employed', 'total_income']].astype(int)
credit_info.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [14]:
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [15]:
# теперь обработаем столбец 'education'
credit_info['education'] = credit_info['education'].str.lower()
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [16]:
# исправим нулевой возраст заемщика
credit_info['dob_years'][credit_info['dob_years'] == 0] = 18
credit_info['dob_years'].min()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


18

In [17]:
# наконец исправим логические неточности в возможной продолжительности стажа в столбце 'days_employed'
max_count_of_days_employed_before = credit_info['days_employed'].max()
print('Максимальный стаж заемщиков до исправления в годах{:9.0f}'.format(max_count_of_days_employed_before / 365))
min_count_of_days_employed_before = credit_info['days_employed'].min()
print('Минимальный стаж заемщиков до исправления в годах{:9.0f}'.format(min_count_of_days_employed_before / 365))
credit_info['days_employed'][credit_info['days_employed'] / 365 > credit_info['dob_years'] -18] = (credit_info['dob_years'] -18) * 365
max_count_of_days_employed_after = credit_info['days_employed'].max()
print('Максимальный стаж заемщиков после исправления в годах{:9.0f}'.format(max_count_of_days_employed_after / 365))
min_count_of_days_employed_after = credit_info['days_employed'].min()
print('Минимальный стаж заемщиков после исправления в годах{:9.0f}'.format(min_count_of_days_employed_after / 365))

Максимальный стаж заемщиков до исправления в годах     1101
Минимальный стаж заемщиков до исправления в годах        0
Максимальный стаж заемщиков после исправления в годах       56
Минимальный стаж заемщиков после исправления в годах        0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


*Отлично, наши манипуляции привели стаж в порядок.*

In [19]:
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5475,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6205,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

1. **Мы убрали отрицательные значения методом abs(), ведь стаж не может быть отрицательным, а также заменили нулевой возраст на минимально возможный.** 
2. **Привели дни стажа и доход к целочислительному типу методом astype().**
3. **Уровень образования "выровнили" методом str.lower().**
4. **Убрали "лишний" опыт из стажа методом присваивания.**
5. **Возможно это были человеческие ошибки при вводе данных, а в случае завышенного стажа манипуляции с целью получения кредита или улучшения условий по нему.**

### Обработка дубликатов

In [20]:
# вычислим количество полных дубликатов после наших манипуляций с данными, но сначала приведем к нижнему регистру все стоблцы с которыми мы еще этого не делали

credit_info['family_status'] = credit_info['family_status'].str.lower()
credit_info['income_type'] = credit_info['income_type'].str.lower()
credit_info['purpose'] = credit_info['purpose'].str.lower()
print('Количество полных дубликатов после наших манипуляций: {}'.format(credit_info.duplicated().sum()))

Количество полных дубликатов после наших манипуляций: 71


*Чтобы выявить все полные дубликаты, их нужно обрабатывать после приведения данных к единому стандарту, иначе результат может отличаться.*

In [21]:
# удалим полные дубликаты

credit_info = credit_info.drop_duplicates().reset_index(drop = True)
print('Количество полных дубликатов после наших манипуляций: {}'.format(credit_info.duplicated().sum()))

Количество полных дубликатов после наших манипуляций: 0


**Вывод**

1. **Мы посчитали количество дубликатов методом "duplicated()" применив к нему метод "sum()".**
2. **Затем удалили дубликаты методом "drop_duplicates()".**
3. **Теперь полных дубликатов у нас нет, возможно они появились при повторных визитах заемщиков для получения займа.**

### Лемматизация

In [22]:
# чтобы выделить суть цели получения кредита применим лемматизацию
from pymystem3 import Mystem
m = Mystem()
        
def lemmatized_purpose(row):
    return ('_'.join(m.lemmatize(row)).replace("_", "")).replace("\n", "")
    
credit_info['lemmatized_purpose'] = credit_info['purpose'].apply(lemmatized_purpose)
credit_info.groupby('lemmatized_purpose') ['lemmatized_purpose'].count()

lemmatized_purpose
автомобиль                                972
высокий образование                       452
дополнительный образование                460
жилье                                     646
заниматься высокий образование            496
заниматься образование                    408
на покупка автомобиль                     471
на покупка подержать автомобиль           478
на покупка свой автомобиль                505
на проведение свадьба                     768
недвижимость                              633
образование                               447
операция с жилье                          652
операция с коммерческий недвижимость      650
операция с недвижимость                   675
операция со свой недвижимость             627
покупка жилой недвижимость                606
покупка жилье                             646
покупка жилье для сдача                   651
покупка жилье для семья                   638
покупка коммерческий недвижимость         661
покупка недвижи

**Вывод**

1. **Для лемматизации мы воспользовались методами "Mystem().lemmatize()" из библиотеки "pymystem3", это эффективнее стемминга т.к. выделяет не просто корень слова, а всю его словарную форму.**
2. **Люди хотят купить/построить/отремонтировать недвижимость, авто, образование, свадьбу и ремонт.**

### Категоризация данных

In [23]:
# теперь опишем свои категории данных в соответствии с заданием исследования
# для ответа на последующие вопросы стоит категоризировать цели кредита и уровень дохода

def category_of_purpose(row):
    if 'автомобиль' in row:
        return 'автомобиль'
    elif 'образование' in row:
        return 'образование'
    elif 'свадьба' in row:
        return 'свадьба'
    elif 'строительство' in row or 'ремонт' in row:
        return 'строительство и ремонт'
    elif 'недвижимость' in row or 'жилье' in row:
        return 'недвижимость'
    else:
        return row
    
credit_info['category_of_purpose'] = credit_info['lemmatized_purpose'].apply(category_of_purpose)
credit_info.groupby('category_of_purpose') ['category_of_purpose'].count()

category_of_purpose
автомобиль                4306
недвижимость              8326
образование               4013
свадьба                   2324
строительство и ремонт    2485
Name: category_of_purpose, dtype: int64

In [24]:
# теперь категоризируем уровень дохода
# чтобы выяснить наши границы доходов рассмотрим их описание в тыс.р. но только для тех кто имеет доход!
(credit_info[credit_info['total_income'] > 0] ['total_income'] / 1000).describe()

count    21454.000000
mean       167.431584
std         98.060605
min         20.667000
25%        107.623000
50%        151.887000
75%        202.417000
max       2265.604000
Name: total_income, dtype: float64

In [25]:
# теперь можно разделить доход по четырем категориям

quantile_25 = credit_info['total_income'].quantile(.25)
quantile_50 = credit_info['total_income'].quantile(.50)
quantile_75 = credit_info['total_income'].quantile(.75)

def category_of_income(row):
    if row == 0:
        return 'доход отсутствует'
    if row < quantile_25:
        return 'низкий'
    elif row < quantile_50:
        return 'ниже среднего'
    elif row < quantile_75:
        return 'выше среднего'
    else:
        return 'высокий'    
    
credit_info['category_of_income'] = credit_info['total_income'].apply(category_of_income)
credit_info.groupby('category_of_income') ['category_of_income'].count()

category_of_income
высокий          5410
выше среднего    5317
ниже среднего    5363
низкий           5364
Name: category_of_income, dtype: int64

**Вывод**

1. **Мы распределили заемщиков по цели кредита (так как в недвижимости не везде было описание коммерческая она или нет (недвижимость, жилье, жилье под сдачу, коммерческая недвижимость, свое жилье и т.д.) чтобы не ошибиться с самостоятельным присваиванием категории коммерческой или своей недвижимости мы всю недвижимость объединили в одну категорию. Также составили категории по уровню дохода в соответствии с распределением которое получили методом "describe()", отсеяли при этом людей без дохода, ведь нам неизвестно какой он может быть потенциально.**
2. **Люди хотят купить/построить/отремонтировать недвижимость, авто, образование, свадьбу и ремонт.**
3. **Средний доход составляет 167 431 р.**
4. **Большинство кредитов связано с недвижимостью, почти поровну идут образование и автомобиль, и замыкает список свадьба.**

## Ответы на вопросы исследования

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
# рассмотрим количество детей в таблице перед построением сводной таблицы

credit_info.groupby('children') ['children'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [27]:
# детей не может быть меньше нуля, скорее всего это ошибка, отсутствие детей между группами 5-20 подозрительно.
# 20 у 76 заемщиков почти невозможно, -1 приведем к 1, а 20 к близкому в данном случае по логике значению 2

credit_info['children'] = credit_info['children'].abs().replace(20, 2)
#credit_info[credit_info['children'] == 20]
credit_info.groupby('children') ['children'].count()

children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
# теперь значения в порядке, можно приступать к проверке платежеспособрности

debt_from_children = pd.pivot_table(credit_info, index = 'children', values = 'debt', aggfunc = 
               lambda x: (x.mean() * 100).round(1)).reset_index()
debt_from_children['count_of_clients'] = credit_info.groupby('children') ['children'].count()
debt_from_children.sort_values(by = ['debt']).style.hide_index()

children,debt,count_of_clients
5,0,9
0,7.5,14091
3,8.2,330
1,9.2,4855
2,9.5,2128
4,9.8,41


In [29]:
# напечатаем на экране то что получилось

for i in debt_from_children.index:
    print('С количеством детей:{0}  доля невозврата = {1}%'.format(
        debt_from_children['children'][i], debt_from_children['debt'][i]))

С количеством детей:0  доля невозврата = 7.5%
С количеством детей:1  доля невозврата = 9.2%
С количеством детей:2  доля невозврата = 9.5%
С количеством детей:3  доля невозврата = 8.2%
С количеством детей:4  доля невозврата = 9.8%
С количеством детей:5  доля невозврата = 0.0%


**Вывод**

1. **Мы проверили процент невозврата по группам от количества детей.**
2. **Группы более 2х не репрезентативны по сравнению с остальными.**
3. **При наличии до 1го ребенка процент невозврата составляет от 7.5 до 9.2 процентов, а вот в группе с двумя детьми 9.5%**
4. **Размах значений доли невозврата по категориям количества детей в самых весомых категориях составляет 2.0%**
5. **Наличие детей влияет на возврат кредита отрицательно.**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
credit_info.groupby('debt') ['debt'].count()

debt
0    19713
1     1741
Name: debt, dtype: int64

In [40]:
debt_from_family_status = pd.pivot_table(credit_info, index = 'family_status', values = 'debt', aggfunc = 
               lambda x: (x.mean() * 100).round(1)).reset_index()
debt_from_family_status = debt_from_family_status.sort_values(by = ['debt']).reset_index(drop = True)
debt_from_family_status.style.hide_index()

family_status,debt
вдовец / вдова,6.6
в разводе,7.1
женат / замужем,7.5
гражданский брак,9.3
не женат / не замужем,9.8


In [32]:
for i in debt_from_family_status.index:
    print('С семейным положением: {0}  доля невозврата = {1}%'.format(
        debt_from_family_status['family_status'][i], debt_from_family_status['debt'][i]))

С семейным положением: вдовец / вдова  доля невозврата = 6.6%
С семейным положением: в разводе  доля невозврата = 7.1%
С семейным положением: женат / замужем  доля невозврата = 7.5%
С семейным положением: гражданский брак  доля невозврата = 9.3%
С семейным положением: не женат / не замужем  доля невозврата = 9.8%


**Вывод**

1. **Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.**
2. **В группах с женатыми/разведенными/овдовевшими процент невозврата находится в диапазоне 6.6 - 7.5%.**
3. **А в группе с гражданским браком 9.3%**
4. **Размах значений доли невозврата по категориям семейного положения составляет 3.2%**
5. **Самые рисковые - не женатые, а самые надежные овдовевшие.**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
debt_from_category_of_income = pd.pivot_table(credit_info, index = 'category_of_income', values = 'debt', aggfunc = 
               lambda x: (x.mean() * 100).round(1)).reset_index()
debt_from_category_of_income = debt_from_category_of_income.sort_values(by = ['debt']).reset_index(drop = True)
debt_from_category_of_income.style.hide_index()

category_of_income,debt
высокий,6.9
низкий,8
выше среднего,8.7
ниже среднего,8.9


In [34]:
for i in debt_from_category_of_income.index:
    print('С доходом типа: {0}  доля невозврата = {1}%'.format(
        debt_from_category_of_income['category_of_income'][i], debt_from_category_of_income['debt'][i]))

С доходом типа: высокий  доля невозврата = 6.9%
С доходом типа: низкий  доля невозврата = 8.0%
С доходом типа: выше среднего  доля невозврата = 8.7%
С доходом типа: ниже среднего  доля невозврата = 8.9%


**Вывод**

1. **Размах значений доли невозврата по категориям дохода составляет 2.0%**
2. **Самая рисковая группа выше среднего и ниже среднего, наименее рисковая с высоким уровнем дохода.**
3. **Без дохода люди лучше платят по кредитам чем со средними доходами.**

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
debt_from_category_of_purpose = pd.pivot_table(credit_info, index = 'category_of_purpose', values = 'debt', aggfunc = 
               lambda x: (x.mean() * 100).round(1)).reset_index()
debt_from_category_of_purpose = debt_from_category_of_purpose.sort_values(by = ['debt']).reset_index(drop = True)
debt_from_category_of_purpose.style.hide_index()

category_of_purpose,debt
недвижимость,7.2
строительство и ремонт,7.2
свадьба,8
образование,9.2
автомобиль,9.4


In [36]:
for i in debt_from_category_of_purpose.index:
    print('С целью типа: {0}  доля невозврата = {1}%'.format(
        debt_from_category_of_purpose['category_of_purpose'][i], debt_from_category_of_purpose['debt'][i]))

С целью типа: недвижимость  доля невозврата = 7.2%
С целью типа: строительство и ремонт  доля невозврата = 7.2%
С целью типа: свадьба  доля невозврата = 8.0%
С целью типа: образование  доля невозврата = 9.2%
С целью типа: автомобиль  доля невозврата = 9.4%


**Вывод**

1. **Размах значений доли невозврата по категориям целей займа составляет 2.2%**
2. **Самая рисковая группа на автомобиль, наименее рисковая недвижимость плюс строительство и ремонт.**

## Общий вывод

**Каждая категория групп клиентов влияет на невозврат долга по своему, ниже представлен общий рейтинг по степени влияния.**

1. **Семейное положение        3.2%**
2. **Цель займа                2.2%**
3. **Дети в весомых категориях 2.0%**
4. **Доход                     2.0%**

**Таким образом Семейное положение оказалось в полтора раза влиятельней цели займа и количества детей, и уровня дохода. Количество детей и доход влияют на возврат одинаково.**